In [1]:
require 'countries'
require 'numo/narray'
require 'nyaplot'

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

true

In [2]:
def sim(v1, v2)
  v1.transpose.dot(v2) / Math.sqrt(v1.transpose.dot(v1) * v2.transpose.dot(v2))
end

def distance(mat, v1)
  result = Hash.new
  mat.shape[0].times do |i|
    v2 = mat[i, true]
    similarity = sim(v1, v2)
    result[i] = similarity
  end

  result.sort{|(_, val1), (_, val2)| val2 <=> val1}[1..10]
end

def analogy(mat, v1, v2, v3)
  result = Hash.new
  v4 = v1 - v2 + v3

  mat.shape[0].times do |i|
    v_ = mat[i, true]
    similarity = sim(v4, v_)
    result[i] = similarity
  end

  result.sort{|(_, val1), (_, val2)| val2 <=> val1}[1..10]
end

:analogy

# 90. word2vecによる学習

81で作成したコーパスに対してword2vecを適用し，単語ベクトルを学習せよ．さらに，学習した単語ベクトルの形式を変換し，86-89のプログラムを動かせ．

In [3]:
unless File.exists?('../data/knock90')
  corpus = Marshal.load(open('../data/knock81.dump'))
  output = open('../data/enwiki_corpus.txt', 'w')
  corpus.each do |document|
    output.puts document
  end
  output.close

  puts 'NOTE: after running this script, please execute following command to create embedding model.'
  puts 'word2vec  -train ../../data/enwiki_corpus.txt -output ../../data/knock90'
  puts 'thanks !'

else

  token2id = Hash.new
  id2token = Hash.new
  mat = nil

  File.foreach('../data/knock90').with_index do |line, index|
    elements = line.chomp.split

    if index == 0
      i, j = elements.map(&:to_i)
      mat = Numo::Float64.zeros(i, j)
      next
    end

    id = index - 1
    token = elements[0]
    vector = elements[1..-1].map(&:to_f)

    token2id[token] = id
    id2token[id] = token

    vector.each_with_index do |xy, d|
      mat[id, d] = xy
    end
  end

end

1

1

In [4]:
mat = mat.to_a

1

1

In [5]:
word_matrix = Numo::Float64.zeros(mat.size, mat[0].size)
mat.each_with_index do |vector, i|
  vector.each_with_index do |x, j|
    word_matrix[i, j] = x
  end
end


vec1 = word_matrix[token2id['United_States'], true]
vec2 = word_matrix[token2id['US'], true]

p 'United_States'
p token2id['United_States']
p vec1
p 'US'
p token2id['US']
p vec2



mat.each_with_index do |vector, i|
  vector.each_with_index do |x, j|
    word_matrix[i, j] = x
  end
end

vec1 = word_matrix[token2id['United_States'], true]
vec2 = word_matrix[token2id['US'], true]

p sim(vec1, vec2)

vec1 = word_matrix[token2id['England'], true]
distance(word_matrix, vec1).map{|id, sim| [id2token[id], sim]}.each do |token, sim|
  puts "#{token}\t#{sim}"
end


vec1 = word_matrix[token2id['Spain'], true]
vec2 = word_matrix[token2id['Madrid'], true]
vec3 = word_matrix[token2id['Athens'], true]

analogy(word_matrix, vec1, vec2, vec3).map{|id, sim| [id2token[id], sim]}.each do |token, sim|
  puts "#{token}\t#{sim}"
end

"United_States"
125
Numo::DFloat(view)#shape=[100]
[-0.458034, 0.111173, -0.032966, 0.261746, 0.301216, -0.280937, -0.07812, ...]
"US"
128
Numo::DFloat(view)#shape=[100]
[-0.282039, 0.158412, -0.026127, 0.216177, 0.267263, -0.246664, -0.239623, ...]
0.7987703599991373
Wales	0.7463648610802359
Scotland	0.6975127954045364
London	0.6457361857345926
Brunswick	0.5687723197471226
Cornwall	0.5676164041591263
Guinea	0.5617349828700244
Australia	0.5591925619141811
Britain	0.5574862359506547
Suffolk	0.5547505069960774
South	0.5504994855423865
Spain	0.5924987016216982
Italy	0.5727125669943777
Austria	0.5512919704168432
Greece	0.5458895408038883
Denmark	0.540896598550964
France	0.5358464125076043
Sweden	0.5300385545000936
Belgium	0.5291697084751801
Netherlands	0.5225235619706168
Germany	0.5157241694180849


[["Spain", 0.5924987016216982], ["Italy", 0.5727125669943777], ["Austria", 0.5512919704168432], ["Greece", 0.5458895408038883], ["Denmark", 0.540896598550964], ["France", 0.5358464125076043], ["Sweden", 0.5300385545000936], ["Belgium", 0.5291697084751801], ["Netherlands", 0.5225235619706168], ["Germany", 0.5157241694180849]]

# 91. アナロジーデータの準備

単語アナロジーの評価データをダウンロードせよ．このデータ中で": "で始まる行はセクション名を表す．例えば，": capital-common-countries"という行は，"capital-common-countries"というセクションの開始を表している．ダウンロードした評価データの中で，"family"というセクションに含まれる評価事例を抜き出してファイルに保存せよ．

In [6]:
problems = []
writable = false

File.foreach('../data/questions-words.txt').map(&:chomp).each do |line|
  if line == ": family"
    writable = true
    next
  end

  if line =~ /^\:/ && writable
    break
  end

  problems << line.split if writable
end

# 92. アナロジーデータへの適用

91で作成した評価データの各事例に対して，vec(2列目の単語) - vec(1列目の単語) + vec(3列目の単語)を計算し，そのベクトルと類似度が最も高い単語と，その類似度を求めよ．求めた単語と類似度は，各事例の末尾に追記せよ．このプログラムを85で作成した単語ベクトル，90で作成した単語ベクトルに対して適用せよ．

In [7]:
def apply(word_matrix, token2id, id2token, problems)
  results = []

  problems.each do |problem|
    next unless token2id[problem[0]] && token2id[problem[1]] && token2id[problem[2]] && token2id[problem[3]]
    vec1 = word_matrix[token2id[problem[0]], true]
    vec2 = word_matrix[token2id[problem[1]], true]
    vec3 = word_matrix[token2id[problem[2]], true]
    vec4 = word_matrix[token2id[problem[3]], true]

    most_similar_id = analogy(word_matrix, vec1, vec2, vec3)[0][0]
    vec4_ = word_matrix[most_similar_id, true]
    similarity = sim(vec4, vec4_)

    puts "#{problem[3]}\t#{id2token[most_similar_id]}\t#{similarity}"
    results << [problem[3], id2token[most_similar_id], similarity]
  end

  results
end


puts "applying start... (it takes few minitutes)"
results = apply(word_matrix, token2id, id2token, problems)

applying start... (it takes few minitutes)
sister	uncle	0.6381235179902043
sisters	brother	0.6493495516718888
mom	distraught	0.7680619738607655
mother	grandfather	0.7179652023407949
grandmother	uncle	0.7784125434717103
granddaughter	nephew	0.7999510856796991
bride	bride	1.0
she	she	1.0
her	His	0.5949219734872246
wife	mother	0.8000473648545017
queen	ruler	0.5127827923682505
woman	boy	0.6838548333684671
niece	uncle	0.7724960707191694
princess	regent	0.6664945180432085
daughter	brother	0.7112107509588852
daughters	daughters	1.0
stepmother	bricklayer	0.660644929405655
stepdaughter	Coton	0.46256929721702544
aunt	grandfather	0.7662448323888026
sisters	brother	0.6493495516718888
mom	dad	0.868332108537061
mother	father	0.8754837318711949
grandmother	uncle	0.7784125434717103
granddaughter	brother	0.6481221207865803
bride	bride	1.0
she	she	1.0
her	His	0.5949219734872246
wife	husband	0.6885193004675382
queen	King	0.4984865075380695
woman	brother	0.2856659467791539
niece	brother	0.6708533066107326

[["sister", "uncle", 0.6381235179902043], ["sisters", "brother", 0.6493495516718888], ["mom", "distraught", 0.7680619738607655], ["mother", "grandfather", 0.7179652023407949], ["grandmother", "uncle", 0.7784125434717103], ["granddaughter", "nephew", 0.7999510856796991], ["bride", "bride", 1.0], ["she", "she", 1.0], ["her", "His", 0.5949219734872246], ["wife", "mother", 0.8000473648545017], ["queen", "ruler", 0.5127827923682505], ["woman", "boy", 0.6838548333684671], ["niece", "uncle", 0.7724960707191694], ["princess", "regent", 0.6664945180432085], ["daughter", "brother", 0.7112107509588852], ["daughters", "daughters", 1.0], ["stepmother", "bricklayer", 0.660644929405655], ["stepdaughter", "Coton", 0.46256929721702544], ["aunt", "grandfather", 0.7662448323888026], ["sisters", "brother", 0.6493495516718888], ["mom", "dad", 0.868332108537061], ["mother", "father", 0.8754837318711949], ["grandmother", "uncle", 0.7784125434717103], ["granddaughter", "brother", 0.6481221207865803], ["bride"

# 93. アナロジータスクの正解率の計算

92で作ったデータを用い，各モデルのアナロジータスクの正解率を求めよ．

In [8]:
result_sims = results.map{|a, b, c| c}
puts "score: #{result_sims.select{|c| c == 1}.size.to_f / result_sims.size}"

score: 0.19473684210526315


# 94. WordSimilarity-353での類似度計算

The WordSimilarity-353 Test Collectionの評価データを入力とし，1列目と2列目の単語の類似度を計算し，各行の末尾に類似度の値を追加するプログラムを作成せよ．このプログラムを85で作成した単語ベクトル，90で作成した単語ベクトルに対して適用せよ．

In [9]:
results = []
File.foreach('../data/combined.tsv') do |line|
  word1, word2, score = line.split
  score = score.to_f
  
  next unless token2id[word1] && token2id[word2]
  similarity = sim(word_matrix[token2id[word1], true], word_matrix[token2id[word2], true])
  puts "#{word1}\t#{word2}\t#{similarity}\t#{score}"
  results << [word1, word2, similarity, score]
end

Word	1	0.2052758335561657	0.0
love	sex	0.4782273730970651	6.77
tiger	cat	0.6268883717514722	7.35
tiger	tiger	1.0	10.0
book	paper	0.4717124093532056	7.46
computer	keyboard	0.3739468506942822	7.62
computer	internet	0.5442778336696681	7.58
plane	car	0.5020497807051268	5.77
train	car	0.5305501679336143	6.31
telephone	communication	0.4453432543369713	7.5
television	radio	0.8149813387686554	6.77
media	radio	0.5151759538483522	7.42
drug	abuse	0.613349458542337	6.85
bread	butter	0.7669784606665079	6.19
cucumber	potato	0.834788388187173	5.92
doctor	nurse	0.48151838412130726	7.0
professor	doctor	0.42008516592869455	6.62
student	professor	0.6078333554108885	6.81
smart	student	0.18632980313692554	4.62
smart	stupid	0.47334365145229523	5.81
company	stock	0.49949832869467303	7.08
stock	market	0.6209558840420333	8.08
stock	phone	0.18500572364394793	1.62
stock	CD	0.023841327913611363	1.31
stock	jaguar	0.000508701987090906	0.92
stock	egg	-0.030889599219515628	1.81
fertility	egg	0.3015535152686933	6.69
s

# 95. WordSimilarity-353での評価

94で作ったデータを用い，各モデルが出力する類似度のランキングと，人間の類似度判定のランキングの間のスピアマン相関係数を計算せよ．

保留

# 96. 国名に関するベクトルの抽出

word2vecの学習結果から，国名に関するベクトルのみを抜き出せ．

In [13]:
countries = ISO3166::Country.all.map{|obj| obj.name.gsub(/\s/, '_')}
countries.each do |country|
  if token2id[country]
    p country
    p word_matrix[token2id[country], true]
  end
end

"Andorra"
Numo::DFloat(view)#shape=[100]
[-0.008513, 0.051645, -0.080665, -0.11307, -0.058117, -0.025352, -0.009851, ...]
"United_Arab_Emirates"
Numo::DFloat(view)#shape=[100]
[-0.086755, 0.023172, -0.110315, 0.101933, 0.027277, -0.033502, -0.004025, ...]
"Afghanistan"
Numo::DFloat(view)#shape=[100]
[-0.18678, 0.261625, -0.219771, 0.039887, 0.045906, -0.242974, -0.224302, ...]
"Albania"
Numo::DFloat(view)#shape=[100]
[-0.113645, 0.468847, -0.058352, 0.235426, -0.00651, 0.061852, 0.015132, ...]
"Armenia"
Numo::DFloat(view)#shape=[100]
[0.018225, 0.026408, -0.391827, -0.149095, -0.142223, 0.068467, -0.090617, ...]
"Angola"
Numo::DFloat(view)#shape=[100]
[-0.045074, 0.222521, -0.139262, -0.021135, 0.097226, 0.037658, 0.088109, ...]
"Antarctica"
Numo::DFloat(view)#shape=[100]
[0.003551, 0.183825, 0.033578, -0.184504, 0.118767, 0.014087, 0.036555, ...]
"Argentina"
Numo::DFloat(view)#shape=[100]
[-0.291491, 0.132292, -0.158745, -0.21848, 0.006782, -0.25224, -0.275714, ...]
"American_Samoa"
N

["Andorra", "United_Arab_Emirates", "Afghanistan", "Antigua_and_Barbuda", "Anguilla", "Albania", "Armenia", "Netherlands_Antilles", "Angola", "Antarctica", "Argentina", "American_Samoa", "Austria", "Australia", "Aruba", "Åland_Islands", "Azerbaijan", "Bosnia_and_Herzegovina", "Barbados", "Bangladesh", "Belgium", "Burkina_Faso", "Bulgaria", "Bahrain", "Burundi", "Benin", "Saint_Barthélemy", "Bermuda", "Brunei_Darussalam", "Bolivia", "Bonaire,_Sint_Eustatius_and_Saba", "Brazil", "Bahamas", "Bhutan", "Bouvet_Island", "Botswana", "Belarus", "Belize", "Canada", "Cocos_(Keeling)_Islands", "Congo,_The_Democratic_Republic_Of_The", "Central_African_Republic", "Congo", "Switzerland", "Côte_D'Ivoire", "Cook_Islands", "Chile", "Cameroon", "China", "Colombia", "Costa_Rica", "Cuba", "Cape_Verde", "Curaçao", "Christmas_Island", "Cyprus", "Czech_Republic", "Germany", "Djibouti", "Denmark", "Dominica", "Dominican_Republic", "Algeria", "Ecuador", "Estonia", "Egypt", "Western_Sahara", "Eritrea", "Spain",

# 97. k-meansクラスタリング

96の単語ベクトルに対して，k-meansクラスタリングをクラスタ数k=5k=5として実行せよ．

In [14]:
# NOTE: we can get data more simply (now, I do not use this procedure because k-means needs to be fixed)
# data = countries.map {|country| token2id[country] ? mat[token2id[country]] : nil }.select {|arr| arr}
# mat = Numo::Float64[*data]

vectors = []
labels = []
num_class = []

countries.each do |country|
  if token2id[country]
    labels << country
    vectors << word_matrix[token2id[country], true]
    num_class << 0
  end
end

# k-means
#
# parameter
k = 5

# initialize
centroids = vectors.shuffle.first(k)

10.times do |num_iter|
  puts "iter: #{num_iter}"

  # assign phase
  vectors.each_with_index do |vector, i|
    similarities = centroids.map do |centroid|
      sim(vector, centroid)
    end

    max_sim = similarities.max
    argmax = similarities.index(max_sim)
    num_class[i] = argmax
  end

  # maximization
  centroids.size.times do |t|
    new_centroid = Numo::Float64.zeros(centroids[0].shape)

    num_points = 0
    vectors.each_with_index do |vector, i|
      if num_class[i] == t
        new_centroid += vector 
        num_points += 1
      end
    end

    centroids[t] = new_centroid / num_points
  end
end

puts

centroids.size.times do |i|
  puts "cluster_#{i}"

  cluster = []
  vectors.size.times do |t|
    cluster << labels[t] if num_class[t] == i
  end

  puts cluster.sort.join(' ')
  puts
end

iter: 0
iter: 1
iter: 2
iter: 3
iter: 4
iter: 5
iter: 6
iter: 7
iter: 8
iter: 9

cluster_0
Bahamas Barbados Bermuda Chad Djibouti Dominican_Republic Ecuador El_Salvador Georgia Grenada Guam Guernsey Guinea Isle_of_Man Jamaica Jersey Jordan Lebanon Mexico Nicaragua Norfolk_Island Oman Panama Puerto_Rico Saint_Lucia

cluster_1
Andorra Bahrain Belarus Bosnia_and_Herzegovina Cameroon Comoros Costa_Rica Croatia Cyprus Czech_Republic Faroe_Islands Ghana Gibraltar Guadeloupe Guatemala Honduras Hungary Kenya Liechtenstein Lithuania Luxembourg Mali Monaco Montenegro Mozambique Nigeria Paraguay Peru Qatar San_Marino Serbia Slovakia Slovenia Trinidad_and_Tobago Tunisia Uruguay Zambia Zimbabwe

cluster_2
Afghanistan Algeria American_Samoa Angola Armenia Azerbaijan Bangladesh Bhutan Bolivia Botswana Bulgaria Cambodia Cayman_Islands Central_African_Republic Congo Curaçao Egypt Equatorial_Guinea Ethiopia Fiji Gambia Guinea-Bissau Haiti Iceland Iraq Kiribati Kuwait Latvia Liberia Libya Macao Malawi Ma

5

# 98. Ward法によるクラスタリング
96の単語ベクトルに対して，Ward法による階層型クラスタリングを実行せよ．さらに，クラスタリング結果をデンドログラムとして可視化せよ．

# 99. t-SNEによる可視化
96の単語ベクトルに対して，ベクトル空間をt-SNEで可視化せよ．

In [15]:
countries = ISO3166::Country.all.map{|obj| obj.name.gsub(/\s/, '_')}
token2id, _, mat = Marshal.load(open('../data/knock90.dump', 'rb'))

data = countries.map {|country| token2id[country] ? mat[token2id[country]] : nil }.select {|arr| arr}
names = countries.map {|country| token2id[country] ? country : nil}.select {|arr| arr}
mat = Numo::Float64[*data]


# TODO: random initialize should be done with normal distribution
tsne_mat = Numo::Float64.new(mat.shape[0], 2).rand

# perp = 0.1
num_iter = 10
eta = 0.01
# momentum = alpha

# compute p_{i, j}
p = Numo::Float64.zeros(mat.shape[0], mat.shape[0])
q = Numo::Float64.zeros(mat.shape[0], mat.shape[0])

# I compute denominator here because denominator of p_{i, j} (appeared in Equation (3)) is constant when X are given.
omega = 1 # TODO: need to compute a variance
denominator = 0
tsne_mat.shape[0].size.times do |k|
  tsne_mat.shape[0].size.times do |l|
    next if k == l
    elem = mat[k, true] - mat[l, true]
    denominator += Math.exp(- elem.transpose.dot(elem) / omega)
  end
end


tsne_mat.shape[0].times do |i|
  tsne_mat.shape[0].times do |j|
    elem1 = mat[i, true] - mat[j, true]
    numerator = Math.exp(- elem1.transpose.dot(elem1) / omega)
    p_j_i = numerator / denominator
    elem2 = mat[j, true] - mat[j, true]
    numerator = Math.exp(- elem2.transpose.dot(elem2) / omega)
    p_i_j = numerator / denominator
    p[i, j] = p_i_j + p_j_i
  end
end


num_iter.times do |t|
  denominator = 0
  tsne_mat.shape[0].size.times do |k|
    tsne_mat.shape[0].size.times do |l|
      next if k == l
      elem = tsne_mat[k, true] - tsne_mat[l, true]
      denominator += 1 + elem.transpose.dot(elem)
    end
  end

  tsne_mat.shape[0].size.times do |i|
    tsne_mat.shape[0].size.times do |j|
      # compute low-dimensional affinities q_{i, j} using Equation 4
      elem1 = tsne_mat[i, true] - tsne_mat[j, true]
      numerator = 1 + elem1.transpose.dot(elem1)

      # NOTE: numerator^{-1} / denominator^{-1}
      q[i, j] = denominator / numerator
    end
  end

  # compute gradient (using Equation 5)
  gradient = Numo::Float64.zeros(tsne_mat.shape)
  tsne_mat.shape[0].times do |i|
    # numerator = 0
    tsne_mat.shape[0].times do |j|
      elem1 = p[i, j] - q[i, j] # scalar
      elem2 = (tsne_mat[i, true] - tsne_mat[j, true]) # vector
      elem3 = 1 / (1 + elem2.transpose.dot(elem2)) # scalar

      dcdy_i = 4 * elem1 * elem2 / elem3

      # TODO: use for loop
      gradient[i, 0] = dcdy_i[0]
      gradient[i, 1] = dcdy_i[1]
    end
  end

  # update
  tsne_mat = tsne_mat + eta * gradient
end


x = tsne_mat[true, 0]
y = tsne_mat[true, 1]

plt = Nyaplot::Plot.new
plt.add(:scatter, x, y)
plt.show()

#<Nyaplot::Frame:0x005641ae6b4f68 @properties={:panes=>[#<Nyaplot::Plot:0x005641ae6f1350 @properties={:diagrams=>[#<Nyaplot::Diagram:0x005641ae6b4680 @properties={:type=>:scatter, :options=>{:x=>"data0", :y=>"data1"}, :data=>"04a3a85d-1b80-47b8-aa62-85029416443d"}, @xrange=[-0.13014120007726385, 1.3785471626877301], @yrange=[-0.525452783027536, 1.0158681580878506]>], :options=>{:zoom=>true, :width=>700, :xrange=>[-0.13014120007726385, 1.3785471626877301], :yrange=>[-0.525452783027536, 1.0158681580878506]}}>], :data=>{"04a3a85d-1b80-47b8-aa62-85029416443d"=>#<Nyaplot::DataFrame:0x005641ae6f2bd8 @name="04a3a85d-1b80-47b8-aa62-85029416443d", @rows=[{:data0=>0.9616160874940824, :data1=>0.765634484474591}, {:data0=>0.6144049317383111, :data1=>0.060993857660704476}, {:data0=>0.3937804327789049, :data1=>0.9258849422579998}, {:data0=>0.23637173720103205, :data1=>0.11114855816707049}, {:data0=>0.31298417354450947, :data1=>0.9594729458944934}, {:data0=>0.6267877563360439, :data1=>0.1771374540584208}, {:data0=>0.8623690560045488, :data1=>0.2316494163132481}, {:data0=>0.24181458202964842, :data1=>0.6009527972753266}, {:data0=>0.789554495220588, :data1=>0.3898804274971877}, {:data0=>0.29062429099035836, :data1=>0.7720161591658943}, {:data0=>0.5168873209758175, :data1=>0.47817538485610744}, {:data0=>1.1839591358531036, :data1=>0.29054387480105237}, {:data0=>0.7536144098316324, :data1=>-0.09950196531090383}, {:data0=>0.6661389669263564, :data1=>-0.03889365150333429}, {:data0=>0.504118822720547, :data1=>0.823183038090706}, {:data0=>0.5343449237059155, :data1=>0.28380771839324564}, {:data0=>0.8793173170209561, :data1=>0.7966305496014358}, {:data0=>0.8459768310757264, :data1=>0.13188122578903919}, {:data0=>0.0381741040669963, :data1=>0.2969821786444604}, {:data0=>0.3333487675535614, :data1=>-0.42411893121362354}, {:data0=>0.2585873890404671, :data1=>1.0131638854620844}, {:data0=>0.017895382011929223, :data1=>-0.08260537960378754}, {:data0=>0.6203581949319011, :data1=>0.1286331460219969}, {:data0=>0.3946590283185546, :data1=>0.2868040544132428}, {:data0=>1.0273472677629554, :data1=>0.42572519346050086}, {:data0=>0.04672182690520615, :data1=>-0.4388724735474092}, {:data0=>0.8192259376179588, :data1=>0.9216527767957059}, {:data0=>0.9467964343540353, :data1=>0.8738750062613755}, {:data0=>0.556830696703363, :data1=>0.10342355305437577}, {:data0=>0.3331777537432736, :data1=>0.7951149964633313}, {:data0=>0.4739229482117666, :data1=>0.12129853753447228}, {:data0=>0.3438051584618179, :data1=>0.27241785089468545}, {:data0=>0.3804054308995531, :data1=>-0.17060699796599685}, {:data0=>0.039672533646020525, :data1=>0.8739953300897614}, {:data0=>0.7860717661304097, :data1=>-0.4598448022720995}, {:data0=>0.17016545907868827, :data1=>-0.19223948457659024}, {:data0=>0.5058457216748222, :data1=>0.6202197623078856}, {:data0=>0.3705889029929463, :data1=>-0.38322154933134395}, {:data0=>1.1582672258405153, :data1=>-0.023233422113100054}, {:data0=>0.0837873383035161, :data1=>0.6478003217740121}, {:data0=>0.37571374636749477, :data1=>0.0684194967891564}, {:data0=>0.37282275053824276, :data1=>0.20971711633078566}, {:data0=>0.5050559462472975, :data1=>0.9243891352528292}, {:data0=>0.5828980754988666, :data1=>-0.03705017004538108}, {:data0=>0.42963415464253213, :data1=>0.9547330781872602}, {:data0=>0.5443946155707035, :data1=>0.710282068095424}, {:data0=>0.4050075568419265, :data1=>0.7307224948673323}, {:data0=>0.5929070940640003, :data1=>0.7435933699770675}, {:data0=>0.8681409869726159, :data1=>0.6530386865347573}, {:data0=>0.5938598488333732, :data1=>0.6850586280849833}, {:data0=>0.9102131367529839, :data1=>0.5837607981702344}, {:data0=>0.9440137401399872, :data1=>-0.04844636512157134}, {:data0=>0.5211052241273894, :data1=>-0.038083724138019776}, {:data0=>1.0057906684867952, :data1=>-0.10432751216349942}, {:data0=>0.24566682453696997, :data1=>0.33047101617202207}, {:data0=>0.6218567717639188, :data1=>0.5383652945373135}, {:data0=>-0.03390423553123732, :data1=>0.79102568